# Exploring the data of Airbnb website. 

#### Student name: Thai Minh Nguyen
#### Student ID: 24002398


# Introduction

[Airbnb](https://www.airbnb.co.nz/) was born in 2007 when two hosts welcomed three guests to their San Francisco home, and has since grown to over 5 million hosts who have welcomed over 1.5 billion guest arrivals in almost every country across the globe. Every day, hosts offer unique stays and experiences that make it possible for guests to connect with communities in a more authentic way. My project is to explore the data of Airbnb website in New Zealand. I will crawl the data from the website, then clean and transform it to a structured format. After that, I will analyze the data to find the relationship between the room features, ratings of listing and host, the price and the weather data.

#### Datasets used:

1. Crawled data from Airbnb website
2. Got the weather data from the API

#### Dataset sources: 

1. https://www.airbnb.co.nz/
2. https://www.visualcrossing.com/

### Research Questions

1. Does the room features affect the price?
2. Does the ratings of listing and host affect the price?
3. Does the weather affect the price?


### Executive Summary

When we book a Airbnb for a holiday, there are many choices for us. We will compare the price of the listing based on the room features, ratings of the listing and host, and the hosting type and guest favorite. The key findings are: the number of bedrooms and baths increases the price of the house, but it can clearly be understood that the price for the bigger room is for more people. The second finding is not logical, when most of the ratings have negative coefficients in the OLS model, while the higher rating should have a higher price. Finally, we see that on the period that has a clear day or is partly cloudy, the price is higher than the period that has a rainy day or cloudy.


In [1]:
from selenium import webdriver
import pandas as pd
import threading
from urllib.parse import urlparse, parse_qs
from datetime import datetime
import glob
import os
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import json
import statsmodels.api as sm

cService = webdriver.ChromeService(executable_path='/Users/benminh1201/Downloads/chromedriver-mac-arm64/chromedriver')

# 1. Data Acquisition

## 1.1. Crawl data from Airbnb

In my project, I use Selenium instead of request and Beautifulsoup because Airbnb use Javascript and API to render the page. I need to use Selenium to open the page and get the data.

In [2]:
# I use 4 threads to crawl data
# 1 page has 15 listings, instead of opening Chrome window 18 times, I use multithreading to open 4 Chrome windows at the same time.
semaphore = threading.Semaphore(4)

In [3]:
# Function use to get all listing url in 1 page. Normally, 1 page has 18 listings.
def get_url_list(url):
    driver = webdriver.Chrome(service=cService)
    driver.get(url)

    craw_list = []

    divs = driver.find_elements(by='class name', value='c4mnd7m')
    for div in divs:
        craw_dict = dict()
        url = div.find_element(by="tag name", value="a").get_attribute('href')
        price = div.find_element(by='class name', value='pquyp1l').text
        craw_dict['URL'] = url
        craw_dict['Price'] = price
        craw_list.append(craw_dict)
    driver.close()

    return craw_list

In [4]:
# Using url finding above to get all listing in 1 page
# This function will get all information of each listing and save to a DataFrame
def get_listings(url):
    semaphore.acquire()
    try:
        listings = pd.DataFrame(
            columns=['URL', 'Title', 'checkin_date', 'Price', 'Area', 'Avg_Rating', 'IsFavorite', 'host_info', 'host_url', 'host_name', 'host_rating', 'features', 'Cleanliness', 'Accuracy', 'Checkin', 'Communication', 'Location', 'Value'])
        
        u = url['URL'] + '&translate_ugc=false'
    
        parsed_url = urlparse(u)
        query_params = parse_qs(parsed_url.query)
        check_in_date = query_params.get('check_in', [None])[0]
        
        driver = webdriver.Chrome(service=cService)
        driver.implicitly_wait(3)
        driver.get(u)


        title = driver.find_element(by="class name", value="_1xxgv6l").text
    
        try:
            ratings = driver.find_elements(by='class name', value='l925rvg')
            Cleanliness = ratings[0].text
            Accuracy = ratings[1].text
            Checkin = ratings[2].text
            Communication = ratings[3].text
            Location = ratings[4].text
            Value = ratings[5].text
        except:
            Cleanliness = "No Rating"
            Accuracy = "No Rating"
            Checkin = "No Rating"
            Communication = "No Rating"
            Location = "No Rating"
            Value = "No Rating"
    
        divs = driver.find_elements(by='class name', value='lgx66tx')
        features = divs[0].text
        host_info = divs[1].text
        
        try:
            areas = driver.find_element(by='class name', value='_leqb4t').text
        except:
            areas = driver.find_element(by='class name', value='_152qbzi').text
    
        try:
            avg_rating = driver.find_element(by='class name', value='gvcwa6y').text
            isFavorite = "Guest Favorite"
        except Exception as e1:
            try:
                avg_rating = driver.find_element(by='class name', value='r1lutz1s').text
                isFavorite = "Not Guest Favorite"
            except Exception as e2:
                avg_rating = "No Rating"
                isFavorite = ""

        hosts = driver.find_elements(by='class name', value='colzjmk')
        for host in hosts:
            host_url = host.find_element(by="tag name", value="a").get_attribute('href')
            
        driver.close()
        
        driver2 = webdriver.Chrome(service=cService)
        driver2.implicitly_wait(3)
        driver2.get(host_url)

        host_name = driver2.find_element(by="class name", value="t1gpcl1t").text
        try:
            host_rating = driver2.find_element(by="class name", value="ruujrrq").text
        except:
            host_rating = None
        
        driver2.close()

        listings = listings._append({
            'URL': u,
            'Title': title,
            'checkin_date': check_in_date,
            'Price': url['Price'],
            'Area': areas,
            'Avg_Rating': avg_rating,
            'IsFavorite': isFavorite,
            'features': features,
            'Cleanliness': Cleanliness,
            'Accuracy': Accuracy,
            'Checkin': Checkin,
            'Communication': Communication,
            'Location': Location,
            'Value': Value,
            'host_info': host_info,
            'host_url': host_url,
            'host_name': host_name,
            'host_rating': host_rating
        }, ignore_index=True)
    
        with results_lock:
            results.append(listings)
    finally:
        semaphore.release()

In [5]:
# Using the above function to get all listings in 1 page
# This function use to split the work to multiple threads
def crawl_and_collect(urls):
    threads = []

    for url in urls:
        thread = threading.Thread(target=get_listings, args=(url,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()
        
    df = pd.concat(results, ignore_index=True)
    return df

In [6]:
# Because the url of each page does not have same pattern, each page has unique id, I cannot use loop to get all pages from the constant url.
# At the end of the page, there is page number and next button.
# The next button use to detect that there is next page or not.
# If there is next page, the function will get the next page url and call itself again.
def find_next_page(url, list_next_page):
    if url == None:
        return list_next_page
    
    list_next_page.append(url)
    
    driver = webdriver.Chrome(service=cService)
    driver.implicitly_wait(3)
    driver.get(url)

    buttonSpace = driver.find_element(by='class name', value='p1j2gy66')
    buttonSingle = buttonSpace.find_elements(by='class name', value='l1ovpqvx')
    for button in buttonSingle:
        if button.get_attribute('aria-label') == "Next":
            nextUrl = button.get_attribute('href')
            driver.close()
            find_next_page(nextUrl, list_next_page)
            
    return list_next_page

In [7]:
# Get all listings in 1 page and merge it as a DataFrame, then save it to a parquet file as historical data, which can be used for analysis later.
# Or if the data have been transformed in the wrong way, we can use this data to re-transform.
def crawler(url, count):
    if url == None:
        return
    
    global results
    results = []
    global results_lock
    results_lock = threading.Lock()
    
    url_list = get_url_list(url)
    
    df_results = crawl_and_collect(url_list)
    
    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    outdir = f'./staging/{year}/{month}/{day}/'
    if not os.path.exists(outdir):
        os.mkdir(outdir)
        
    # I choose to use parquet file because it is faster than csv file and optimize for columnar storage.
    df_results.to_parquet(outdir + f'page{count}.parquet', engine='pyarrow')

Demo method to get listing of page 1

In [8]:
page_1 = "https://www.airbnb.co.nz/s/New-Zealand/homes?tab_id=home_tab&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-06-01&monthly_length=3&monthly_end_date=2024-09-01&price_filter_input_type=0&channel=EXPLORE&query=New%20Zealand&place_id=ChIJh5Z3Fw4gLG0RM0dqdeIY1rE&location_bb=weg99sMvjGvCU0cVQyWOaw%3D%3D&date_picker_type=calendar&adults=1&source=structured_search_input_header&search_type=autocomplete_click"

crawler(page_1, 1)

![Demo](/Users/benminh1201/Downloads/SCR-20240528-nfbk.png)

In [ ]:
page_1 = "https://www.airbnb.co.nz/s/New-Zealand/homes?tab_id=home_tab&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-06-01&monthly_length=3&monthly_end_date=2024-09-01&price_filter_input_type=0&channel=EXPLORE&query=New%20Zealand&place_id=ChIJh5Z3Fw4gLG0RM0dqdeIY1rE&location_bb=weg99sMvjGvCU0cVQyWOaw%3D%3D&date_picker_type=calendar&adults=1&source=structured_search_input_header&search_type=autocomplete_click"

list_next_page = find_next_page(page_1, [])

In order to save time of running get url of all pages, I save it to a csv file at the first running, which can use to run the function later.

In [10]:
list_next_page = pd.read_csv('staging/2024/5/26/list_next_page.csv')['0'].tolist()
list_next_page

I split it to run in 3 times. Because I need to change VPN to access the website after I crawl many times.

In [ ]:
count = 0
for url in list_next_page[:5]:
    count += 1
    crawler(url, count)

In [ ]:
for url in list_next_page[5:11]:
    count += 1
    crawler(url, count)

In [ ]:
for url in list_next_page[11:]:
    count += 1
    crawler(url, count)

After running the function, I have full 15 pages in staging layer which save raw data.

![listfile](/Users/benminh1201/Downloads/SCR-20240528-nhxh.png)

## 1.2. Crawl weather data from api

I only get the weather data of city and date, which is the same as the checkin date and area of the listing. So the csv file here is the data which is transformed in the Data Wrangling step. The weather data which I got is the date of the previous year of the checkin date.

In [11]:
# There can be the listing which has the same checkin date and city, so I need to drop duplicates.
df = pd.read_csv('foudation/listings.csv')
list_area = df[['Area', 'checkin_date']].copy().drop_duplicates(subset=['Area', 'checkin_date'])

In [12]:
# I get the weather in city and date of the previous year of the checkin date.
list_area['checkin_date'] = pd.to_datetime(list_area['checkin_date'])
list_area['fake_checkin_date'] =  list_area['checkin_date'] - pd.DateOffset(years=1)

In [13]:
def get_weather(city, date):
    response = requests.request("GET", f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/{date}?unitGroup=metric&include=days&key=GKHM62FQSWEDNTBQEQJ3M2UBC&contentType&contentType=json")
    if response.status_code!=200:
        return {
            'tempmax': None,
            'tempmin': None,
            'temp': None,
            'humidity': None,
            'main': None
        }

    result = json.loads(response.content)
    return {
        'tempmax': result['days'][0]['tempmax'],
        'tempmin': result['days'][0]['tempmin'],
        'temp': result['days'][0]['temp'],
        'humidity': result['days'][0]['humidity'],
        'main': result['days'][0]['icon']
    }

In [14]:
# Demo of function
get_weather('Auckland', '2023-01-12')

Link of api: https://www.visualcrossing.com/

I find the weather of the previous year of the checkin date.

In [15]:
list_area['weather'] = list_area.apply(lambda x: get_weather(x['Area'], x['fake_checkin_date'].strftime('%Y-%m-%d')), axis=1)
list_area

In [16]:
# Splitting the dictionary to columns, then save to a parquet file.
list_area = list_area.join(pd.json_normalize(list_area['weather']))
list_area

In [18]:
now = datetime.now()
year = datetime.now().year
month = datetime.now().month
day = datetime.now().day

list_area.to_parquet(f'./staging/{year}/{month}/{day}/weather.parquet', engine='pyarrow')

# 2. Data Wrangling

In [21]:
path = f'staging/2024/5/26/'
all_files = glob.glob(os.path.join(path, "*.parquet"))
df_list = [pd.read_parquet(file) for file in all_files]
df = pd.concat(df_list, ignore_index=True)
df.shape

There are 269 listings in the dataset.

In [22]:
df.head()

### 2.2. Data cleaning

#### 2.2.1 Price

There are 3 columns related to price: OriginalPrice, DiscountPrice, and TotalPrice. We will split the Price column into 3 columns: OriginalPrice, DiscountPrice, and TotalPrice (Price for 5 days). We will also remove the currency symbol and convert the price to float.

In [23]:
df['OriginalPrice'] = df['Price'].str.split('\n', expand=True)[0].replace('', None)
df['DiscountPrice'] = df['Price'].str.split('\n', expand=True)[1]
df['TotalPrice'] = df['Price'].str.split('\n', expand=True)[5]
df.drop(columns=['Price'], inplace=True)
df.head(10)

In [24]:
df['DiscountPrice'] = df['DiscountPrice'].replace('night', None)
df['DiscountPrice'] = df['DiscountPrice'].str.replace('$', '').str.replace(' NZD', '').astype(float)

df['OriginalPrice'] = df['OriginalPrice'].str.replace('$', '').str.replace(' NZD', '').astype(float)

df['TotalPrice'] = df['TotalPrice'].str.split(' ', expand=True)[0]
df['TotalPrice'] = df['TotalPrice'].str.replace('$', '').str.replace(',', '').astype(float)

df.head(10)

#### 2.2.2 Area

I only keep the city name in the Area column. There are some listings with the area name as 'NZ' or 'Au', which I will replace with 'Auckland'.

In [25]:
df['Area'] = df['Area'].str.split(', ').apply(lambda x: x[-2])
df['Area'] = df['Area'].replace('NZ', 'Auckland')
df['Area'] = df['Area'].replace('Au', 'Auckland')
df.head(10)

#### 2.2.3 Average Rating

I split the rating from the sentence and convert it to float. Some listings have a new rating, which I will replace with -1.

In [26]:
df['Avg_Rating'] = df['Avg_Rating'].str.replace('New', '-1')
df['Avg_Rating'] = df['Avg_Rating'].str.split(' ').apply(lambda x: x[1] if len(x) > 1 else x[0]).astype(float)
df.head(10)

#### 2.2.4 Host Information

The original host_info contains the host type (Superhost) and hosting time. I will split this column into 2 columns: hosting_type and hosting_time. I will also convert the host_rating to float.

In [27]:
df['host_info'] = df['host_info'].apply(lambda x: "None · " + x if "·" not in x else x)
df['hosting_type'] = df['host_info'].str.split('·', expand=True)[0].str.strip()
df['hosting_time'] = df['host_info'].str.split('·', expand=True)[1]
df['host_rating'] = df['host_rating'].astype(float)
df.drop(columns=['host_info'], inplace=True)
df

#### 2.2.5 Rating

On Airbnb, there are 6 types of ratings: cleanliness, accuracy, check-in, communication, location, and value. I will split the rating from the sentence and convert it to float.

In [28]:
df['Cleanliness'] = df['Cleanliness'].str.split('\n').apply(lambda x: x[-1] if len(x) > 1 else None).astype(float)
df['Accuracy'] = df['Accuracy'].str.split('\n').apply(lambda x: x[-1] if len(x) > 1 else None).astype(float)
df['Checkin'] = df['Checkin'].str.split('\n').apply(lambda x: x[-1] if len(x) > 1 else None).astype(float)
df['Communication'] = df['Communication'].str.split('\n').apply(lambda x: x[-1] if len(x) > 1 else None).astype(float)
df['Location'] = df['Location'].str.split('\n').apply(lambda x: x[-1] if len(x) > 1 else None).astype(float)
df['Value'] = df['Value'].str.split('\n').apply(lambda x: x[-1] if len(x) > 1 else None).astype(float)
df

#### 2.2.6 Features

The features of each listing are in different formats. This listing may have this feature and may not have other feature. Thus,  I need to find the unique features and keep only the features that are common in most listings.

In [29]:
df['features'] = (df['features']
                  .str.replace('beds', 'bed')
                  .str.replace('bedrooms', 'bedroom')
                  .str.replace('baths', 'bath')
                  .str.replace('guests', 'guest'))
df_temp = df['features'].str.split('·', expand=True).melt()
df_temp

Because the data of feaures column does not have a consistent format, I reshape the data to 1 column to find the unique feature by using regex to remove the number in string.

The features can have "1 bath" or "1.5 baths", so I use regex to remove the number in the string.

In [30]:
df_temp['value'].replace('\d+\.?\d*', '', regex=True).str.strip().value_counts()

From the above table, we can see that the features which account for most of the data is number of guest, bath, bedroom and bed. Thus, I only keep these features. I also keep the studio option.

In [31]:
def extract_details(details):
    parts = details.split(' · ')
    guest, bath, bed, bedroom, studio = None, None, None, None, 0

    for part in parts:
        if 'guest' in part:
            guest = float(part.split(' ')[0])
        elif 'bath' in part and 'Half' not in part and 'shared' not in part and 'Shared' not in part and 'private' not in part and 'Private' not in part and 'Dedicated' not in part:
            bath = float(part.split(' ')[0])
        elif 'bedroom' in part:
            bedroom = float(part.split(' ')[0])
        elif 'bed' in part:
            bed = float(part.split(' ')[0])
        elif 'Studio' in part:
            studio = 1

    return guest, bath, bed, bedroom, studio

df[['guest', 'bath', 'bed', 'bedroom', 'studio']]= df['features'].apply(lambda x: pd.Series(extract_details(x)))
df.drop(columns=['features'], inplace=True)
df

#### 2.2.7 Hosting time

In [32]:
df[df['hosting_time'].str.contains('month')]['hosting_time']

Because the data has both year and month, I will convert month to year by dividing by 12.

In [33]:
df['hosting_time'] = df['hosting_time'].str.replace('hosting', '').str.strip()

def transform_duration(time):
    years = 0
    months = 0

    if 'New Host' in time:
        return 0
    if 'year' in time:
        years = int(time.split(' ')[0])
    if 'month' in time:
        months_str = time.split(' ')[0]
        months = int(months_str) / 12

    return years + months

df['hosting_time'] = df['hosting_time'].apply(transform_duration)

#### 2.2.8 Get listing and host ID

I assume that the sequence number in the URL is the unique id

In [34]:
df['host_id'] = df['host_url'].str.split('/', expand=True)[5]
df['listing_id'] = df['URL'].str.split('/', expand=True)[4].str.split('?', expand=True)[0]

Save transformed data to a csv file. In assignment 3, I will use this csv file to insert to SQlite database.

In [35]:
df.to_csv('foudation/listings.csv', index=False)

In [36]:
df

In the step 1.2, I have already transformed the weather data, so I will use it to merge with the listing data.

In [37]:
df_weather = pd.read_parquet('staging/2024/5/26/weather.parquet')
df_weather

In [41]:
df['checkin_date'] = pd.to_datetime(df['checkin_date'])

In [42]:
df_full = pd.merge(df, df_weather, on=['Area', 'checkin_date'], how='left')
df_full.shape

In [43]:
df_full

### 2.3. Exploratory Data Analysis

In [44]:
feq=df['Area'].value_counts().sort_values(ascending=True)
feq.plot.barh(figsize=(8, 8), width=0.8)
plt.title("Number of listings by Area", fontsize=20)
plt.xlabel('Number of listings', fontsize=12)

plt.show()

The most listings are in Waikato, followed by Auckland and Northland.

In [45]:
feq=df['guest'].value_counts().sort_index()
feq.plot.bar(figsize=(10, 8), color='b', width=0.8, rot=0)
plt.title("Maximum guest per listings", fontsize=20)
plt.ylabel('Number of listings', fontsize=12)
plt.xlabel('Accommodates', fontsize=12)
plt.show()

Most of the listings allow maximum 2 guests.

In [46]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)

feq=df['bath'].value_counts().sort_index()
feq.plot.bar(color='b', width=0.8, rot=0)
plt.ylabel('Number of listings', fontsize=12)
plt.title('Number of bath per listings')

plt.subplot(1, 3, 2)

feq=df['bed'].value_counts().sort_index()
feq.plot.bar(color='b', width=0.8, rot=0)
plt.ylabel('Number of listings', fontsize=12)
plt.title('Number of bed per listings')

plt.subplot(1, 3, 3)

feq=df['bedroom'].value_counts().sort_index()
feq.plot.bar(color='b', width=0.8, rot=0)
plt.ylabel('Number of listings', fontsize=12)
plt.title('Number of bathroom per listings')

plt.show()

There is typically one bedroom, one bathroom, and one bed.

In [47]:
feq=df['studio'].value_counts().sort_index()
feq.plot.bar(color='b', width=0.8, rot=0)
plt.ylabel('Number of listings', fontsize=12)
plt.title('Distribution of Studio and Non-Studio Listings')

Not many listings has the studio.

In [54]:
plt.hist(df['Avg_Rating'], color='b')
plt.ylabel('Frequency', fontsize=12)
plt.xlabel('Average rating', fontsize=12)
plt.title("Histogram of Average rating", fontsize=12)

Most of the ratings are between 3.8 to 5. Minus 1 rating is the new listing.

In [86]:
feq = df.groupby('Area')['TotalPrice'].mean().sort_values(ascending=True)
feq.plot.barh(figsize=(8, 8), color='b', width=0.8)
plt.title("Average 5-night price for 1-person accommodation", fontsize=20)
plt.xlabel('Average 5-night price (NZD)', fontsize=12)
plt.show()

Because the default number of guests when I crawl the data is for 1 adult in 5 nights. Thus, the price is for 1 person.
On average, the price is the highest in Otago, followed by Marlborough Sounds and Golden Bay.

In [55]:
fig = plt.figure(figsize=(20,15))
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)

ax1 = fig.add_subplot(321)
feq=df['Location'].value_counts().sort_index()
ax1=feq.plot.bar(color='b', width=0.8, rot=0)
#ax1.tick_params(axis = 'both', labelsize = 16)
plt.title("Location", fontsize=24)
plt.ylabel('Number of listings', fontsize=14)
plt.xlabel('Average review score', fontsize=14)

ax2 = fig.add_subplot(322)
feq=df['Cleanliness'].value_counts().sort_index()
ax2=feq.plot.bar(color='b', width=0.8, rot=0)
plt.title("Cleanliness", fontsize=24)
plt.ylabel('Number of listings', fontsize=14)
plt.xlabel('Average review score', fontsize=14)

ax3 = fig.add_subplot(323)
feq=df['Value'].value_counts().sort_index()
ax3=feq.plot.bar(color='b', width=0.8, rot=0)
plt.title("Value", fontsize=24)
plt.ylabel('Number of listings', fontsize=14)
plt.xlabel('Average review score', fontsize=14)

ax4 = fig.add_subplot(324)
feq=df['Communication'].value_counts().sort_index()
ax4=feq.plot.bar(color='b', width=0.8, rot=0)
plt.title("Communication", fontsize=24)
plt.ylabel('Number of listings', fontsize=14)
plt.xlabel('Average review score', fontsize=14)

ax5 = fig.add_subplot(325)
feq=df['Checkin'].value_counts().sort_index()
ax5=feq.plot.bar(color='b', width=0.8, rot=0)
plt.title("Arrival", fontsize=24)
plt.ylabel('Number of listings', fontsize=14)
plt.xlabel('Average review score', fontsize=14)

ax6 = fig.add_subplot(326)
feq=df['Accuracy'].value_counts().sort_index()
ax6=feq.plot.bar(color='b', width=0.8, rot=0)
plt.title("Accuracy", fontsize=24)
plt.ylabel('Number of listings', fontsize=14)
plt.xlabel('Average review score', fontsize=14)

plt.tight_layout()
plt.show()

Similar to the overall rating, most of the ratings are between larger than 4.

In [56]:
# Create the first subplot
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
feq=df['hosting_type'].value_counts()
feq.plot.bar(width=0.8, rot=0)
plt.title("Number of listings with Superhost", fontsize=20)
plt.ylabel('Number of listings', fontsize=12)

# Create the second subplot
plt.subplot(1, 2, 2)
feq=df['IsFavorite'].value_counts()
feq.plot.bar(width=0.8, rot=0)
plt.title("Number of listings with Guest Favorite", fontsize=20)
plt.ylabel('Number of listings', fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

There is 2 kind of host: Superhost and Regular host, and Guest favorite or not. Overall, Superhost and Guest Favorite are more than the other.

In [57]:
feq=df[['host_id', 'host_name']].value_counts().sort_values(ascending=False)[:20]
feq
feq.plot.bar(width=0.8, rot=90)
plt.title("Number of listings per host", fontsize=20)
plt.ylabel('Number of listings', fontsize=12)

Because the data is only show the first or last name of the host, so I need to group by id and name.
Daniel is the host with the most listings, with 6.

### 2.4. Filling missing values

In [60]:
msno.bar(df, color='b')

The Discount price has the most missing values, because not all listings have discount price. I fill the missing values with the original price.

In [61]:
df['DiscountPrice'] = df['DiscountPrice'].fillna(df['OriginalPrice'])
df['guest'] = df['guest'].fillna(2)
df['bath'] = df['bath'].fillna(df['guest'] / 2)
df['bed'] = df['bed'].fillna(df['guest'] / 2)
df['bedroom'] = df['bedroom'].fillna(df['guest'] / 2)

I assume that maximum guest for each listing is 2, so I fill the missing values of bath, bed, and bedroom with half of the maximum guest.

### 2.5. Analysis

#### Does the room features affect the price?

In [62]:
correlation_matrix = df[['OriginalPrice', 'bed', 'bedroom', 'bath', 'guest', 'studio']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

The price has a positive correlation with the number of maximum guest, beds, bedrooms, and bath, but a negative correlation with the number of studio. However, the correlation is not strong.

In [63]:
df_ols = df[['OriginalPrice', 'bed', 'bedroom', 'bath', 'guest', 'studio']].dropna()
# Prepare the data
X = df_ols[['bed', 'bedroom', 'bath', 'guest', 'studio']]
y = df_ols['OriginalPrice']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

# Print the model summary
print(model.summary())

The R-squared value is 0.4, which means that the model explains 40% of the variance in the data.
The number of bedrooms and bath are significant predictors of price, with more bedrooms and bathrooms associated with higher prices, because they have high positive coefficient and small p-value.

#### Does the ratings of listing and host affect the price?

In [64]:
correlation_matrix2 = df[['OriginalPrice', 'Avg_Rating', 'host_rating', 'Cleanliness', 'Accuracy', 'Checkin', 'Communication', 'Location', 'Value']].corr()

sns.heatmap(correlation_matrix2, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

Overall, the correlation of price with ratings is not strong.

In [65]:
df_ols2 = df[['OriginalPrice', 'Avg_Rating', 'host_rating', 'Cleanliness', 'Accuracy', 'Checkin', 'Communication', 'Location', 'Value']].dropna()
# Prepare the data
X = df_ols2[['Avg_Rating', 'host_rating', 'Cleanliness', 'Accuracy', 'Checkin', 'Communication', 'Location', 'Value']]
y = df_ols2['OriginalPrice']

X = sm.add_constant(X)
# Fit the model
model = sm.OLS(y, X).fit()

# Print the model summary
print(model.summary())

The R-squared value is just 0.3. The important variable is Average Rating, Cleanliness, Location and Value with small p-value. However, we see the negative coefficient of Cleanliness and Value variables, which is not make sense.

I convert the hosting type and guest favorite to binary variable, then use them to predict the price.

In [66]:
df['hosting_type_binary'] = df['hosting_type'].str.strip().apply(lambda x: 1 if x == 'Superhost' else 0)
df['IsFavorite_binary'] = df['IsFavorite'].apply(lambda x: 1 if x == 'Guest Favorite' else 0)

In [67]:
df_ols3 = df[['OriginalPrice', 'Avg_Rating', 'host_rating', 'Cleanliness', 'Accuracy', 'Checkin', 'Communication', 'Location', 'Value', 'hosting_type_binary', 'IsFavorite_binary', 'hosting_time']].dropna()
# Prepare the data
X = df_ols3[['Avg_Rating', 'host_rating', 'Cleanliness', 'Accuracy', 'Checkin', 'Communication', 'Location', 'Value', 'hosting_type_binary', 'IsFavorite_binary', 'hosting_time']]
y = df_ols3['OriginalPrice']

X = sm.add_constant(X)
# Fit the model
model = sm.OLS(y, X).fit()

# Print the model summary
print(model.summary())

The R-squared is not increase. And we see most coefficient is negative, while all variables is rating, in real life, the higher rating should have higher price.

#### Does weather affect the price?

In [69]:
correlation_matrix3 = df_full[['OriginalPrice', 'tempmax', 'tempmin', 'temp', 'humidity']].corr()

sns.heatmap(correlation_matrix3, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [84]:
df_ols4 = df_full[['OriginalPrice', 'tempmax', 'tempmin', 'temp', 'humidity', 'main']].dropna()
df_ols4 = pd.get_dummies(df_ols4, columns=['main'])
# Prepare the data
X = df_ols4[['tempmax', 'tempmin', 'temp', 'humidity', 'main_clear-day', 'main_cloudy', 'main_partly-cloudy-day', 'main_rain']]
y = df_ols4['OriginalPrice']

X = sm.add_constant(X)
# Fit the model
model = sm.OLS(y, X.astype(float)).fit()

# Print the model summary
print(model.summary())

The clear day and partly cloudy day have high positive coefficient, which mean that the price is higher in these days. 